# Comparison checks for quality control flags
Covers QC8-15

## Table of contents
[QC8 R99p](#QC8---R99p)  
[QC9 PRCPTOT](#QC9---PRCPTOT)  
[QC10 World Record](#QC10---World-Record)  
[QC11 Rx1day](#QC11---Rx1day)  
[QC12 CDD](#QC12---CDD)  
[QC13 Daily accumulations](#QC13---Daily-accumulations)  
[QC14 Monthly accumulations](#QC14---Monthly-accumulations)  
[QC15 Streaks](#QC15---Streaks)  

See '3.3.1 Suspiciously high values' in Lewis et al. (2021)

# QC8 - R99p 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC9 - PRCPTOT 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC10 - World Record 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC11 - Rx1day 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC12 - CDD 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC13 - Daily accumulations
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

In [ ]:
        vals = list(self.gauge.data)

        mean_wet_day_val, mean_wet_day_val_filled = self.get_sdii()

        flags = [0 for i in range(len(vals))]

        for i in range(len(vals) - 24):
            day_val_list = vals[i:i + 24]
            flag = utils.daily_accums_day_check(day_val_list, mean_wet_day_val, mean_wet_day_val_filled)
            if flag > max(flags[i:i + 24]):
                flags[i:i + 24] = [flag for j in range(24)]

        return flags


 -->   utils.daily_accums_day_check(day_list, mean_wet_day_val, mean_wet_day_val_filled):
    """
    Suspect daily accumulations flagged where a recorded rainfall amount at these times is preceded by 23 hours with no
    rain. A threshold of 2x the mean wet day amount for the corresponding month is applied to increase the chance of
    identifying accumulated values at the expense of genuine, moderate events.
    """

    if day_list[23] > 0:
        dry_hours = 0
        for i in range(23):
            if day_list[i] <= 0:
                dry_hours += 1
        if dry_hours == 23:
            if np.isnan(mean_wet_day_val):
                if day_list[23] > mean_wet_day_val_filled:
                    flag = 2
                else:
                    flag = 0
            else:
                if day_list[23] > mean_wet_day_val:
                    flag = 1
                else:
                    flag = 0
        else:
            flag = 0
    else:
        flag = 0

    return flag

# QC14 - Monthly accumulations 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC15 - Streaks 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 